### Generación del corpus

In [1]:
p1 = """El sol se alza cada mañana sobre el horizonte, pintando el cielo de tonos anaranjados y dorados.
Los pájaros despiertan su canto y el mundo renace con la luz de un nuevo día.
Aprovecha cada amanecer como una invitación a crear algo valioso."""
p2 = """En el corazón del bosque, el susurro del viento recorre las copas de los árboles.
Las hojas bailan al compás de una sinfonía natural, mientras los rayos de luz se filtran entre las ramas.
Caminar por ese sendero es encontrarse con la esencia pura de la vida."""
p3 = """La lluvia cae suave sobre los campos recién sembrados, nutriendo la tierra sedienta.
Cada gota es un poema que riega esperanzas y alimenta sueños ocultos bajo el suelo.
Después de la tormenta, surge un arcoíris que nos recuerda la belleza de la calma tras la tempestad."""

corpus = "\n".join([p1, p2, p3])

### Tokenización y conteo de N-grams

In [2]:
import re
from collections import Counter
import math

def tokenize(text):
    text = text.lower()
    return re.findall(r"\b\w+\b", text)

def build_ngrams(tokens, n):
    return [" ".join(tokens[i:i+n]) for i in range(len(tokens)-n+1)]

tokens = tokenize(corpus)
unigrams = Counter(tokens)
bigrams = Counter(build_ngrams(tokens, 2))
trigrams = Counter(build_ngrams(tokens, 3))

print("Top 5 Unigrams:", unigrams.most_common(5))
print("Top 5 Bigrams:", bigrams.most_common(5))
print("Top 5 Trigrams:", trigrams.most_common(5))

Top 5 Unigrams: [('la', 9), ('de', 8), ('el', 7), ('los', 4), ('cada', 3)]
Top 5 Bigrams: [('de la', 3), ('con la', 2), ('el sol', 1), ('sol se', 1), ('se alza', 1)]
Top 5 Trigrams: [('el sol se', 1), ('sol se alza', 1), ('se alza cada', 1), ('alza cada mañana', 1), ('cada mañana sobre', 1)]


### Cálculo de entropía y perplejidad en mini-corpus

In [3]:
mini_corpus = ["a a b", "a b a"]
mini_tokens = [t for sent in mini_corpus for t in tokenize(sent)]
mini_unigrams = Counter(mini_tokens)
total = sum(mini_unigrams.values())
probs = {w: c/total for w, c in mini_unigrams.items()}

entropy = -sum(p * math.log2(p) for p in probs.values())
perplexity = 2 ** entropy

print(f"\nMini-corpus Unigram Probabilities: {probs}")
print(f"Entropía: {entropy:.4f} bits")
print(f"Perplejidad: {perplexity:.4f}")


Mini-corpus Unigram Probabilities: {'a': 0.6666666666666666, 'b': 0.3333333333333333}
Entropía: 0.9183 bits
Perplejidad: 1.8899


### Smoothing Laplace (Add-1) para bigramas

In [4]:
V = len(unigrams)
def laplace_prob(bigram):
    w1, w2 = bigram.split()
    return (bigrams.get(bigram, 0) + 1) / (unigrams[w1] + V)

example = "pájaros despiertan"
print(f"\nP_Laplace({example}): {laplace_prob(example):.4f}")


P_Laplace(pájaros despiertan): 0.0206


### Smoothing Good–Turing para bigramas

In [5]:
freq_of_freq = Counter(bigrams.values())
N = sum(bigrams.values())
C_star = {}
for r, Nr in freq_of_freq.items():
    Nr1 = freq_of_freq.get(r+1, 0)
    C_star[r] = (r+1)*Nr1/Nr if Nr > 0 else 0
P0 = freq_of_freq.get(1, 0) / N

print("\nGood-Turing adjusted counts C* for some r:")
for r in sorted(list(C_star))[:5]:
    print(f" r={r}  C*={C_star[r]:.4f}")
print(f"P0 (prob. unseen): {P0:.4f}")


Good-Turing adjusted counts C* for some r:
 r=1  C*=0.0154
 r=2  C*=3.0000
 r=3  C*=0.0000
P0 (prob. unseen): 0.9630
